In [1]:
from lxml import etree
import re
from tqdm.notebook import tqdm
from itertools import combinations
import os

class Model(object):
    def __init__(self, corpus, n=2, model="vanilla", replace=False):
        x_counts = corpus.NGram(n,model=model,replace=replace)['gram']

        probabilities = {}
        if n is not 1:
            previous = corpus.NGram(n-1,model=model,replace=replace)['gram']
            for x in x_counts:
                probabilities[x] = {"probability":x_counts[x]["count"] / previous[x[1:]]["count"]}
        else:
            N = len([w for s in corpus for w in s])
            for x in x_counts:
                probabilities[x] = {"probability":x_counts[x]["count"] / N}

        self.probabilities = probabilities
        self.model = model

    def GetProbability(self, forX, givenY:tuple):
        # Add input validation
        sequence = tuple(forX) + givenY

        if sequence in self.probabilities:
            return self.probabilities[sequence]["probability"]
        else:
            return 0

class Corpus(object):
    def __init__(self, corpus=None, directory="Corpus/"):
        if corpus is None:
            self._corpus = self._CorpusAsListOfSentences(directory)
        else:
            self._corpus = corpus
        self._ngrams = {}
        self._ngrams[1] = self.NGram(n=1)
        self._models = {}
        self._models[1] = self.CreateModel(n=1)

    def __len__(self):
        return len(self._corpus)

    def __iter__(self):
        for word in self._corpus:
            yield word

    def __getitem__(self, index):
        return self._corpus[index]

    # Create functions to free memory once function scope is left
    @staticmethod
    def _readCorpus(root="Corpus/"):
        if not os.access(root, os.R_OK):
            print("Check root!!")

        xml_data = []

        for file in tqdm(os.listdir(root), desc='Reading Files'):
            xml_data.append(open(os.path.join(root, file), 'r', encoding='utf8').read())  # Read file
        return xml_data

    @staticmethod
    def _ParseAsXML(root="Corpus/"):
        parser = etree.XMLParser(recover=True)
        roots = []
        xml_data = Corpus._readCorpus(root)
        for xml in tqdm(xml_data, desc='Parsing XML'):
            roots.append(etree.fromstring(xml, parser=parser))
        return roots

    # Mention garbage collection effort and try outs for pd and numpy and why i settled with list of lists (pd matrix, list of lists no)
    @staticmethod
    def _CorpusAsListOfSentences(root="Corpus/"):
        roots = Corpus._ParseAsXML(root)
        sentences = []
        for root in tqdm(roots, desc='XML File'):
            for i, p in tqdm(enumerate(root), desc='Paragraph'):
                for k, s in enumerate(p):
                    unfiltered_sentence = re.split(r'\n', s.text.lstrip('\n'))
                    sentence = []
                    for unfiltered_word in unfiltered_sentence:
                        if unfiltered_word is not "":
                            filtered_word = unfiltered_word.split('\t')
                            sentence.append(filtered_word[0])
                    if sentence is not []:
                        sentence.insert(0, "<s>")
                        sentences.append(sentence)
                        sentence.append("</s>")
        return sentences

    def X_Counts(self, n, laplace=0):
        x_counts = {}
        for s in tqdm(self, desc='Counting x counts'):
            for i in range(len(s) + 1):
                if i < n - 1 or i == 0:
                    continue
                count = []
                for x in range(n,0,-1):
                    count.append(s[i - x])
                count = tuple(count)

                if count in x_counts:
                    x_counts[count] += 1
                else:
                    x_counts[count] = 1
        if laplace == 1:
            for x in x_counts:
                x_counts[x] += 1
        return x_counts

    def NGram(self, n=2, model="vanilla", replace=False):
        if n < 1:
            raise Exception("Unigrams and up are supported, otherwise no.")

        if model != "vanilla" and \
            model != "laplace" and \
            model != "unk":
            raise Exception("Only 'vanilla'/'laplace'/'unk' models are supported.")

        if not replace:
            if n in self._ngrams:
                if  self._ngrams[n]["flags"]["model"] == model:
                    return self._ngrams[n]
            if n not in self._ngrams:
                replace = True


        laplace = 0

        if model == "laplace":
            laplace = 1
        x_counts = self.X_Counts(n=n, laplace=laplace)

        if model == "unk":
            _count = self.X_Counts(1)
            tc = []
            for s in self:
                ts = []
                for w in s:
                    if _count[tuple([w])] < 3:
                        ts.append("UNK")
                    else:
                        ts.append(w)
                tc.append(ts)

            temp = Corpus(corpus=tc)
            x_counts = temp.X_Counts(n=n, laplace=laplace)


        gram = {}
        for x in x_counts:
            gram[x] = {"count":x_counts[x]}

        result =    {   "gram":gram,
                        "flags":
                        {
                            "model": model
                        }
                    }

        if replace:
            self._ngrams[n] = result
            return self._ngrams[n]
        else:
            return result

    def CreateModel(self, n=2, model="vanilla", replace=False):
        if n in self._models:
            if  self._models[n].model == model:
                return self._models[n]
        if n not in self._models:
            replace = True

        if replace:
            self._models[n] = Model(corpus=self,n=n,model=model,replace=replace)
            return self._models[n]
        else:
            return Model(corpus=self,n=n,model=model)


#http://pages.di.unipi.it/pibiri/papers/NGrams18.pdf

corpus = Corpus(directory="Test Corpus/")
#
#
#
# bigram = corpus.NGram()
#
#
#
# trigram = corpus.NGram(3)




vanilla = corpus.CreateModel()
#
#
#laplace = corpus.CreateModel(model="laplace",replace=True)
#
#
#
#
unk = corpus.CreateModel(model="unk")
# y = tuple(["b"])
# print(laplace.GetProbability("c",y))

# TODO: Create Model Object

Reading Files:   0%|          | 0/1 [00:00<?, ?it/s]

Parsing XML:   0%|          | 0/1 [00:00<?, ?it/s]

XML File:   0%|          | 0/1 [00:00<?, ?it/s]

Paragraph: 0it [00:00, ?it/s]

Counting x counts:   0%|          | 0/1 [00:00<?, ?it/s]

...

Counting x counts:   0%|          | 0/1 [00:00<?, ?it/s]

Counting x counts:   0%|          | 0/1 [00:00<?, ?it/s]

Counting x counts:   0%|          | 0/1 [00:00<?, ?it/s]

Counting x counts:   0%|          | 0/1 [00:00<?, ?it/s]

Counting x counts:   0%|          | 0/1 [00:00<?, ?it/s]